This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/2023.2.1_prerelease/wallaroo-features/pipeline_multiple_replicas_forecast_tutorial).

## Statsmodel Forecast with Wallaroo Features: Deploy and Test Infer

This tutorial series demonstrates how to use Wallaroo to create a Statsmodel forecasting model based on bike rentals.  This tutorial series is broken down into the following:

* Create and Train the Model:  This first notebook shows how the model is trained from existing data.
* Deploy and Sample Inference:  With the model developed, we will deploy it into Wallaroo and perform a sample inference.
* Parallel Infer:  A sample of multiple weeks of data will be retrieved and submitted as an asynchronous parallel inference.  The results will be collected and uploaded to a sample database.
* External Connection:  A sample data connection to Google BigQuery to retrieve input data and store the results in a table.
* ML Workload Orchestration:  Take all of the previous steps and automate the request into a single Wallaroo ML Workload Orchestration.

In the previous step "Statsmodel Forecast with Wallaroo Features: Model Creation", the statsmodel was trained and saved to the Python file `forecast.py`.  This file will now be uploaded to a Wallaroo instance as a Python model, then used for sample inferences.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials: Inference Guide: Parallel Inferences](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-inferences/#parallel-inferences)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries that we will need.

In [ ]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
wallaroo.__version__

### Initialize connection

Start a connect to the Wallaroo instance and save the connection into the variable `wl`.

In [ ]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

Workspace names must be unique.  To allow this tutorial to run in the same Wallaroo instance for multiple users, the `suffix` variable is generated from a random set of 4 ASCII characters.  To use the same workspace each time, hard code `suffix` and verify the workspace name created is is unique across the Wallaroo instance.

In [ ]:
# used for unique connection names

import string
import random

suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

workspace_name = f'multiple-replica-forecast-tutorial-{suffix}'
pipeline_name = 'bikedaypipe'
model_name = 'bikedaymodel-standard'

### Set the Workspace and Pipeline

The workspace will be either used or created if it does not exist, along with the pipeline.

In [ ]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload Model

The Python model created in "Forecast and Parallel Infer with Statsmodel: Model Creation" will now be uploaded.  Note that the Framework and runtime are set to `python`.

In [88]:
model_file_name = 'forecast_standard.py'

bike_day_model = wl.upload_model(model_name, model_file_name, Framework.PYTHON).configure(runtime="python")

### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.  The pipeline configuration will allow for multiple replicas of the pipeline to be deployed and spooled up in the cluster.  Each pipeline replica will use 0.25 cpu and 512 Gi RAM.

In [89]:
# Set the deployment to allow for additional engines to run
pipeline.undeploy()
pipeline.clear()
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(0.25)
                        .memory("512Mi")
                        .build()
                    )

pipeline.add_model_step(bike_day_model)
# pipeline.add_model_step(step)
pipeline.deploy(deployment_config = deploy_config)

name,bikedaypipe
created,2023-07-25 18:31:52.188885+00:00
last_updated,2023-07-25 22:11:16.276236+00:00
deployed,True
tags,
versions,"13ef1c91-e3f4-4a89-b4a8-2699bf78a99d, d78a152c-7fcf-4ea8-b7b0-06c9dc6dc085, 872ef2ea-bbc5-4c21-8264-479510101a04, 39510b6e-039c-4b02-a044-31995d34ac33, 39f66adc-5526-4f59-8222-e6f5a0bb2a8d, 631f83e0-0725-452d-b7bf-a6371f9a74cf, 70c98e44-f89c-456d-b22b-f3f0a6e49892, 8ba06289-673e-487b-b99a-70e981d00e50, 48bbf6e0-9f41-48a3-a1b2-cd7ca910b94c, ccab4bac-6f56-47a2-9098-980831565d10, b51dd2ae-583d-41b9-a617-bf99bd42bf90"
steps,bikedaymodel-standard


### Run Inference

Run a test inference to verify the pipeline is operational from the sample test data stored in `./data/testdata_dict.json`.

In [ ]:
# inferencedata = pd.read_json('./data/testdata_standard.df.json', orient="records")

inferencedata = json.load(open("./data/testdata_dict.json"))

results = pipeline.infer(inferencedata)

display(results)

In [91]:
inferencedata = pd.read_json('./data/testdata_standard.df.json')
display(inferencedata)
results = pipeline.infer(inferencedata)

# display(results)

,cnt
0,"[1526, 1550, 1708, 1005, 1623, 1712, 1530, 1605, 1538, 1746, 1472, 1589, 1913, 1815, 2115, 2475, 2927, 1635, 1812, 1107, 1450, 1917, 1807, 1461, 1969, 2402, 1446, 1851]"


InferenceError: Inference failed: 
	InternalError: There was an error sending data to the pipeline: Received none result for Model Inference

In [73]:
step = wl.upload_model("step", "./step.py", Framework.PYTHON).configure(runtime="python")
# pipeline.add_model_step(step)
# pipeline.deploy()

In [ ]:
pipeline.steps()

In [ ]:
pipeline.clear()
pipeline.add_model_step(bike_day_model).deploy(deployment_config = deploy_config)

In [ ]:
pipeline.steps()

In [ ]:
results = pipeline.infer(inferencedata)

display(results)

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [ ]:
pipeline.undeploy()